In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (115 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

# Show DataFrame
home_sales_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
#Data Cleaning to split the date column into the year, month, and day
from pyspark.sql.functions import split, col
split_date = split(home_sales_df['date'], '-')
home_sales_df = home_sales_df.withColumn('year', split_date.getItem(0))
home_sales_df = home_sales_df.withColumn('month', split_date.getItem(1))
home_sales_df = home_sales_df.withColumn('day', split_date.getItem(2))

home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|   04| 08|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|   06| 13|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|   04| 12|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|   10| 16|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878

In [5]:
# 2. Create a temporary view of the DataFrame.
sales = home_sales_df.createOrReplaceTempView('sales')


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
four_bedroom_df = spark.sql("SELECT * FROM sales WHERE bedrooms = 4")
four_bedroom_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|   04| 08|
|f233cb41-6f33-4b0...|2021-07-18|      2016|437375|       4|        3|       1704|   11721|     2|         0|  34|2021|   07| 18|
|2dc662fe-57b8-44b...|2020-04-20|      2016|417215|       4|        2|       2104|    8227|     2|         0|  48|2020|   04| 20|
|d5f2062a-8950-417...|2019-09-02|      2016|305980|       4|        2|       1799|   13004|     1|         0|  26|2019|   09| 02|
|f5e01433-f7b0-44e...|2021-03-10|      2010|335326|       4|        3|       1588|   14107

In [7]:
# 3. Create a temporary view of the DataFrame.
four_bedroom = four_bedroom_df.createOrReplaceTempView('four_bedroom')

In [9]:
# 3. Using the four_bedroom_df sorted dataframe, the average price of the house will be calculated per year
spark.sql("""SELECT year, round(mean(price),2) FROM four_bedroom
              Group By year Order By year desc """).show()


+----+---------------------+
|year|round(mean(price), 2)|
+----+---------------------+
|2022|            296363.88|
|2021|            301819.44|
|2020|            298353.78|
|2019|             300263.7|
+----+---------------------+



In [10]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

three_bedroom_df = spark.sql("SELECT * from sales WHERE bedrooms = 3 AND bathrooms = 3")
three_bedroom_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|2020|   06| 16|
|2ed8d509-7372-46d...|2021-08-06|      2015|258710|       3|        3|       1918|    9666|     1|         0|  25|2021|   08| 06|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|2019|   02| 27|
|941bad30-eb49-4a7...|2020-05-09|      2015|229896|       3|        3|       2197|    8641|     1|         0|   3|2020|   05| 09|
|ea620c7b-c2f7-4c6...|2021-05-31|      2011|437958|       3|        3|       2356|   11052

In [11]:
#Create a temporary view for the three bedroom and three bathroom dataframe
three_bedroom = three_bedroom_df.createOrReplaceTempView('three_bedroom')

In [12]:
# 4. Calculating the average home price from the three_bedroom_df and 3 bathrooms by the year the house was built
spark.sql("""SELECT date_built, round(mean(price),2) FROM three_bedroom
              Group By date_built Order By date_built desc """).show()

+----------+---------------------+
|date_built|round(mean(price), 2)|
+----------+---------------------+
|      2017|            292676.79|
|      2016|            290555.07|
|      2015|             288770.3|
|      2014|            290852.27|
|      2013|            295962.27|
|      2012|            293683.19|
|      2011|            291117.47|
|      2010|            292859.62|
+----------+---------------------+



In [13]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
three_bed_two_floors = spark.sql("SELECT * from three_bedroom WHERE floors=2 AND sqft_living>=2000")
three_bed_two_floors.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|2020|   06| 16|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|2019|   02| 27|
|4566cd2a-ac6e-435...|2019-07-15|      2016|177541|       3|        3|       2130|   10517|     2|         0|  25|2019|   07| 15|
|e0fc52aa-c349-4ba...|2019-04-17|      2016|202790|       3|        3|       2025|   10987|     2|         0|  19|2019|   04| 17|
|e252c4ce-b5b2-4ef...|2019-10-28|      2014|168463|       3|        3|       2271|    8317

In [14]:
#Create a temporary view for the three bedroom and three bathroom dataframe
three_bedroom_two_floors = three_bed_two_floors.createOrReplaceTempView('three_bed_two_floors')

In [15]:
#Average price of homes that have 3 bedrooms, 3 bathrooms, 2 floors, and greater than or equal to 2000 sqaure feet by date built

spark.sql("""SELECT date_built, round(mean(price),2) FROM three_bed_two_floors
              Group By date_built Order By date_built desc """).show()

+----------+---------------------+
|date_built|round(mean(price), 2)|
+----------+---------------------+
|      2017|            280317.58|
|      2016|             293965.1|
|      2015|            297609.97|
|      2014|            298264.72|
|      2013|            303676.79|
|      2012|            307539.97|
|      2011|            276553.81|
|      2010|            285010.22|
+----------+---------------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 5.459785461425781e-05 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data


In [ ]:
# 11. Read the parquet formatted data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

